In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.datasets.adult import Adult
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki
from FairRanking.models.DirectRanker import DirectRanker
from FairRanking.models.BaseDirectRanker import convert_data_to_tensors
from FairRanking.models.DirectRankerAdv import DirectRankerAdv
from FairRanking.TrainingFunctions import DirectRankerAdvTrain, DirectRankerTrain
from FairRanking.helpers import rND_torch, nDCG_cls_no_model, auc_estimator, group_pairwise_accuracy, auc_estimator2, nDCG_cls

In [2]:
path = '../Results/DirectRankerAdv/Adult/Schedule05/results.csv'
with open(path, 'w') as file:
    file.write(f'nDCG,AUC,rND,GPA,Schedule\n')
def write_results(model, X, y, s, schedule, path):
    model.eval()
    with torch.no_grad():
        pred = model.predict_proba(X)
        ndcg_list = []
        for _ in range(100):
            ndcg_i = nDCG_cls(pred, y, esti=False, trec=False, at=500)
            ndcg_list.append(ndcg_i)
        ndcg = np.mean(ndcg_list)
        auc = auc_estimator2(pred, y)
        rnd = rND_torch(pred, torch.argmax(s, dim=1))
        gpa = group_pairwise_accuracy(pred, y, s)
        with open(path, 'a') as file:
            file.write(f'{ndcg},{auc},{rnd},{gpa},{str(schedule).replace(",",";")}\n')


In [7]:
#data = Law('Gender','/Users/robert/Desktop/Bachelor/FairRanker/data')
data = Adult('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Compas('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Wiki()
full_data = data.get_data()
n_epochs = 1000
hidden_layers = [64, 32, 16]
bias_layers = [64, 32, 16]
main_lr = 0.01
adv_lr = 0.01
schedules = [[i,j] for j in range(1,8) for i in range(1,8)]
for schedule in schedules:
        torch.manual_seed(42)
        model = DirectRankerAdv(num_features=full_data[0][0].shape[1],
                        kernel_initializer=nn.init.normal_,
                        hidden_layers=hidden_layers,
                        bias_layers=bias_layers,
                        num_classes=len(torch.unique(full_data[0][1])),
                )
        model = DirectRankerAdvTrain.train(model, full_data, n_epochs=n_epochs, schedule=schedule, threshold=0.0, adv_lr=adv_lr, main_lr=main_lr, print_res=False)
        write_results(model, full_data[2][0], full_data[2][1], full_data[2][2], schedule, path)
        print(f'Finished Schedule: {schedule}')
print('Finished Scheduling')


model = DirectRankerAdv(num_features=full_data[0][0].shape[1],
                kernel_initializer=nn.init.normal_,
                hidden_layers=hidden_layers,
        )
model = DirectRankerAdvTrain.train(model, full_data, n_epochs=1000, main_lr=main_lr, print_res=False, schedule=[1,0])
write_results(model, full_data[2][0], full_data[2][1], full_data[2][2], 'BaseDR', path)

Finished Schedule: [1, 1]
Finished Schedule: [2, 1]
Finished Schedule: [3, 1]
Finished Schedule: [4, 1]
Finished Schedule: [5, 1]
Finished Schedule: [6, 1]
Finished Schedule: [7, 1]
Finished Schedule: [1, 2]
Finished Schedule: [2, 2]
Finished Schedule: [3, 2]
Finished Schedule: [4, 2]
Finished Schedule: [5, 2]
Finished Schedule: [6, 2]
Finished Schedule: [7, 2]
Finished Schedule: [1, 3]
Finished Schedule: [2, 3]
Finished Schedule: [3, 3]
Finished Schedule: [4, 3]
Finished Schedule: [5, 3]
Finished Schedule: [6, 3]
Finished Schedule: [7, 3]
Finished Schedule: [1, 4]
Finished Schedule: [2, 4]
Finished Schedule: [3, 4]
Finished Schedule: [4, 4]
Finished Schedule: [5, 4]
Finished Schedule: [6, 4]
Finished Schedule: [7, 4]
Finished Schedule: [1, 5]
Finished Schedule: [2, 5]
Finished Schedule: [3, 5]
Finished Schedule: [4, 5]
Finished Schedule: [5, 5]
Finished Schedule: [6, 5]
Finished Schedule: [7, 5]
Finished Schedule: [1, 6]
Finished Schedule: [2, 6]
Finished Schedule: [3, 6]
Finished Sch

In [5]:
idx = 2
data = Law('Race','/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Adult('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Compas('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Wiki()#
full_data = data.get_data()
model = DirectRankerAdv(num_features=full_data[0][0].shape[1],
                kernel_initializer=nn.init.normal_,
                hidden_layers=[64, 32, 16],
        )
model = DirectRankerAdvTrain.train(model, full_data, n_epochs=1000, main_lr=0.01, print_res=True, schedule=[1,0])
model.eval()
with torch.no_grad():
        pred = model.predict_proba(full_data[idx][0])
        pred = model(full_data[idx][0], torch.zeros_like(full_data[idx][0], dtype=torch.float32))
        ndcg = nDCG_cls_no_model(pred, full_data[idx][1], esti=False, trec=False, at=500)
        auc = auc_estimator(pred, full_data[idx][1])
        rnd = rND_torch(pred, torch.argmax(full_data[idx][2], dim=1))
        gpa = group_pairwise_accuracy(pred, full_data[idx][1], full_data[idx][2])
        ndcg2 = nDCG_cls(pred, full_data[idx][1], esti=False, trec=False, at=500)

        print(f'{ndcg}, {ndcg2}, {auc}\n')

Loss: 1.173614263534546
Loss: 0.1691061407327652
Loss: 0.07065233588218689
Loss: 0.05756095424294472
Loss: 0.03538850322365761
Loss: 0.025657476857304573
Loss: 0.023794008418917656
Loss: 0.01738457940518856
Loss: 0.019351957365870476
Loss: 0.012602664530277252
0.8127822279930115, 0.8156078991084409, 0.4974169048626522



In [6]:
X = full_data[2][0]
y = full_data[2][1]
s = full_data[2][2]
model.eval()
with torch.no_grad():
    pred = model.predict_proba(X)
    ndcg_list = []
    for _ in range(100):
        ndcg_i = nDCG_cls(pred, y, esti=False, trec=False, at=500)
        ndcg_list.append(ndcg_i)
    ndcg = np.mean(ndcg_list)
    auc = auc_estimator2(pred, y)
    rnd = rND_torch(pred, torch.argmax(s, dim=1))
    gpa = group_pairwise_accuracy(pred, y, s)
print(f'{ndcg}\t{auc}\t{rnd}\t{gpa}')

0.8157980994524344	0.056419683257918574	0.27219955704800775	0.004233642225682788
